In [36]:
from graphics import *
import random
import numpy as np
import time

### Create 5 * 5 grid world environment

In [37]:
win = GraphWin("My window", 600,600)
win.setBackground(color_rgb(255,255,255))

In [38]:
def line(x1,y1,x2,y2):
    """Create a line between point(x1,y1) and point(x2,y2)"""
    
    ln = Line(Point(x1,y1), Point(x2,y2))
    ln.setOutline(color_rgb(0,0,0))
    ln.setWidth(2)
    
    return ln

### border line

In [39]:
border1 = line(50,50,550,50)
border1.draw(win)

border2 = line(550,50,550,550)
border2.draw(win)

border3 = line(50,550, 550,550)
border3.draw(win)

border4 = line(50,50,50,550)
border4.draw(win)

Line(Point(50.0, 50.0), Point(50.0, 550.0))

In [40]:
# Horizontal line
ln1 = line(50,150,550,150)
ln1.draw(win)

ln2 = line(50,250,550,250)
ln2.draw(win)

ln3 = line(50,350,550,350)
ln3.draw(win)

ln4 = line(50,450,550,450)
ln4.draw(win)

Line(Point(50.0, 450.0), Point(550.0, 450.0))

In [41]:
#Vertical line
lnv1 = line(150,50,150,550)
lnv1.draw(win)

lnv2 = line(250,50,250,550)
lnv2.draw(win)

lnv3 = line(350,50,350,550)
lnv3.draw(win)

lnv4 = line(450,50,450,550)
lnv4.draw(win)

Line(Point(450.0, 50.0), Point(450.0, 550.0))

In [42]:
# set goal, hole, obstacle

rect_goal = Rectangle(Point(450,50), Point(550,150))
rect_goal.setFill(color_rgb(0,255,0))
rect_goal.draw(win)

rect_hole = Rectangle(Point(450,150), Point(550,250))
rect_hole.setFill(color_rgb(255,0,0))
rect_hole.draw(win)

rect_obs1 = Rectangle(Point(150,250), Point(250,350))
rect_obs1.setFill(color_rgb(0,0,0))
rect_obs1.draw(win)

rect_obs2 = Rectangle(Point(150,350), Point(250,450))
rect_obs2.setFill(color_rgb(0,0,0))
rect_obs2.draw(win)

Rectangle(Point(150.0, 350.0), Point(250.0, 450.0))

### Create agent look like as circle

In [43]:
agent = Circle(Point(100,500),25)
agent.setFill(color_rgb(0,0,255))
agent.draw(win)

Circle(Point(100.0, 500.0), 25)

### define action

In [44]:
def up(agent):
    val = agent.getCenter()
    if (val.getY()) == 100 or (val.getY() == 500 and val.getX() == 200):
        pass
    else:
        agent.move(0,-100)
        
def down(agent):
    val = agent.getCenter()
    if val.getY() == 500 or (val.getX() == 200 and val.getY() == 200):
        pass
    else:
        agent.move(0, 100)
        
def right(agent):
    val = agent.getCenter()
    if val.getX() == 500 or (val.getX() == 100 and (val.getY() == 300 or val.getY() == 400)):
        pass
    else:
        agent.move(100,0)
        
def left(agent):
    val = agent.getCenter()
    if val.getX() == 100 or (val.getX() == 300 or (val.getY() == 300 or val.getY() == 400)):
        pass
    else:
        agent.move(-100,0)

In [45]:
def action(agent, val):
    if val == 0:
        up(agent)
    elif val == 1:
        down(agent)
    elif val == 2:
        right(agent)
    else:
        left(agent)

In [46]:
for i in range(200):
    val = random.choice([0,1,2,3])
    action(agent,val)

In [47]:
reward = np.array([[-1,-1,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-20,-1,-1,-1],
                   [-1,-1,-1,-1,-100],
                   [-1,-1,-1,-1,100]])

In [48]:
def rewards(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100)
    
    return reward[i, j]

In [49]:
def observation(agent):
    val = agent.getCenter()
    i = int(val.getX() / 100) - 1
    j = int((500 - val.getY()) / 100)
    
    return [i,j]

In [50]:
def env_close():
    win.close()

In [51]:
def reset(agent):
    agent.move(-400,400)

In [52]:
def set_agent(agent, obs):
    pos = agent.getCenter()
    
    pos = [pos.getX(), pos.getY()]
    
    refer_pos = [(obs[0]+1) * 100, (500 - (obs[1]*100))]
    
    agent.move(refer_pos[0]-pos[0], refer_pos[1]-pos[1])

### Initialize the value function and policy

In [53]:
old_value = np.array([[.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,100.0]])

new_value = np.array([[.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,.0],
                      [.0,.0,.0,.0,100.0]])

In [54]:
#policy assign random policy
policy = np.zeros([5,5])

for row in range(5):
    for col in range(5):
        policy[row, col] = random.choice([0,1,2,3])

In [55]:
# declare the variable
policy_iteration = 100
policy_evaluation = 1
change_policy_after_every_episode = 50
discount_factor = 0.9
learning_rate = 0.01

In [58]:
for j in range(policy_iteration):
    #change policy after every 5 iteration and start value with zero
    if j != 100:
        #improve the policy depends upon maximum value of neighbour
        acti = [0,1,2,3]
        for row in range(5):
            for col in range(5):
                maxi = 0
                maxi_action = 0
                for act in acti:
                    set_agent(agent,[row,col])
                    action(agent,act)
                    
                    obser = observation(agent)
                    #select action gives maximum value neighbour
                    if old_value[obser[0],obser[1]] > maxi:
                        maxi_action = act
                        maxi = new_value[obser[0], obser[1]]
                policy[row,col] = maxi_action
    
    for i in range(policy_evaluation):
        for row in range(5):
            for col in range(5):
                set_agent(agent,[row,col])
                Reward = rewards(agent)
                #choose the action
                act = policy[row, col]
                
                action(agent,act)
                new_state = observation(agent)
                
                new_value[row, col] = Reward + (discount_factor *old_value[new_state[0],new_state[1]])
        
        new_value[4,4] = 100
        old_value = new_value

In [61]:
policy

array([[0., 0., 0., 2., 1.],
       [2., 0., 0., 2., 1.],
       [2., 0., 0., 2., 1.],
       [0., 0., 0., 2., 2.],
       [0., 0., 0., 0., 0.]])

In [63]:
new_value

array([[ 37.3513931,  42.612659 ,  48.45851  ,  54.9539   ,  48.45851  ],
       [ 42.612659 ,  29.45851  ,  54.9539   ,  62.171    ,  54.9539   ],
       [ 48.45851  ,  35.9539   ,  62.171    ,  70.19     ,  62.171    ],
       [ 54.9539   ,  62.171    ,  70.19     ,  79.1      , -10.       ],
       [ 62.171    ,  70.19     ,  79.1      ,  89.       , 100.       ]])

In [64]:
old_value

array([[ 37.3513931,  42.612659 ,  48.45851  ,  54.9539   ,  48.45851  ],
       [ 42.612659 ,  29.45851  ,  54.9539   ,  62.171    ,  54.9539   ],
       [ 48.45851  ,  35.9539   ,  62.171    ,  70.19     ,  62.171    ],
       [ 54.9539   ,  62.171    ,  70.19     ,  79.1      , -10.       ],
       [ 62.171    ,  70.19     ,  79.1      ,  89.       , 100.       ]])